In [1]:
import json
import networkx as nx
import dynetx as dn
import re
import numpy as np
from scipy.stats import poisson

In [51]:
def understand() :
    # G = nx.Graph()
    # G.add_edge(2, 3)
    # G.add_edge(1, 2)

    DG = dn.DynGraph()
    # DG.add_interactions_from(G.edges(), t=0)
    DG.add_interactions_from([(1,3)],t=0)
    DG.add_interactions_from([(1,2)],t=1)
    DG.add_interactions_from([(1,3)],t=1)


    DG.add_interactions_from([(1,2)],t=2)
    DG.add_interactions_from([(2,3)],t=2)
    DG.add_interactions_from([(1,3)],t=2)

    DG.add_interactions_from([(1,3)],t=3)

    DG.add_interactions_from([(1,4)],t=4)
    print(DG.interactions([1,3]))

    # G = nx.Graph([(edge[0], edge[1]) for edge in DG.interactions(t=0)])
    # print(nx.degree(G))


    # print(DG.time_slice(t_from=0, t_to=4).nodes())
    # print(DG.time_slice(t_from=0, t_to=1).degree())
    

    # for e in DG.stream_interactions():
    #     print(e)
    
    # print(DG.time_slice(t_from=0, t_to=2).edges)
    # print(DG.time_slice(3).edges)
    # g = nx.MultiGraph(DG.time_slice(t_from=0, t_to=2).edges)
    # print(g.edges())
    # nx.draw_circular(g, with_labels=True)
    
understand()

[(1, 3, {'t': [[0, 3]]}), (1, 2, {'t': [[1, 2]]}), (1, 4, {'t': [[4, 4]]}), (3, 2, {'t': [[2, 2]]})]


In [3]:
def readPreferenceGraph(filePath) :
    with open("./preferenceGraph.json") as file :
        data = json.load(file)

    G = nx.DiGraph()

    for edge in data["edges"] :
        G.add_edge(edge["source"], edge["target"], comment=edge["comment"])
    
    return G

In [149]:
def interactionGraph(preferenceGraph, personalityThreshold, maxInteractions=None) :
   def analyzePreferenceGraph(timestamp) :
      def calculateProbability(weight) :
         if re.match(r".*prefers.*collaborate.*|.*prefers.*socialize.*", weight) :
               return 1
         elif re.match(r".*perceives.*prefers.*collaborate.*|.*perceives.*prefers.*socialize.*", weight) :
            return 0.7
         elif re.match(r".*perceives.*avoids.*collaborate.*|.*perceives.*avoids.*socialize.*", weight) :
            return 0.3
         else :
            return 0
      
      #dictionary {edge : probability of existing in the Preference Graph}
      edgesProbabilities = {}

      #examine each eage in Preference Graph
      #calculate probability
      for u, v in preferenceGraph.edges() :
         if preferenceGraph.has_edge(v, u) :
            #(u,v) and (v,u) are checked together once
            if (v,u) in edgesProbabilities :
                #(v,u) is already checked, so neglect (u,v)
               continue 
            else :
               #(u,v), (v,u) pair has not been checked
               edgesProbabilities[(u,v)] = 0.5*(calculateProbability(preferenceGraph[u][v]["comment"]) + calculateProbability(preferenceGraph[v][u]["comment"]))
         else :
            edgesProbabilities[(u,v)] = calculateProbability(preferenceGraph[u][v]["comment"])

      #create edge in Interaction Graph with respect to the existing edge probability
      for edge, prob in edgesProbabilities.items() :
         if np.random.choice([0,1], p=[1-prob, prob]) == 1 :
            interactionGraph.add_interaction(edge[0], edge[1], timestamp)
      
      return edgesProbabilities
      
      
   def analyzeStudentsPreferences(timestamp) :

      #get a snapshot at the timestamp where students' preferences were added      
      #calculate degree of nodes
      snapshotDegrees = interactionGraph.degree(t=timestamp)

      #detect the maximum degree
      maxDegree = max(snapshotDegrees.values())

      #find the ratio of a node over the maximum
      for node, degree in snapshotDegrees.items() :
         snapshotDegrees[node] = degree/maxDegree
      
      #split students in low and high profile based on a threshold
      lowProfileStudents = set()
      highProfileStudents = set()
      for node, degree in snapshotDegrees.items() :
         if degree < personalityThreshold :
            lowProfileStudents.add(node)
         else :
            highProfileStudents.add(node)
      
      #distribution of each students' category
      lowProfileDistribution = poisson.pmf(range(5), mu=0.8)
      highProfileDistribution = poisson.pmf(range(5), mu=1.5)


      return lowProfileStudents, highProfileStudents, checkProbSum(lowProfileDistribution), checkProbSum(highProfileDistribution)

   def checkProbSum(arr) :
      diff = 1-sum(arr)
      if diff > 0 :
         arr[-1] += abs(diff)
      elif diff < 0 :
         arr[-1] -= abs(diff)
      else :
         None
      return arr


   def createRandomEdgesAtTimestamp(timestamp) :
      nodes = preferenceGraph.nodes()
      for node in nodes :
         
         newInteractions = []
         for other in nodes :
            if ((node, other) in edgesProbabilities and edgesProbabilities[(node, other)] == 0) \
               or ((other, node) in edgesProbabilities and edgesProbabilities[(other, node)] == 0) \
               or other == node :
               continue
            newInteractions.append(other) 
         print(len(newInteractions))
               

         profileDistribution = lowProfileDistribution if node in lowProfileStudents else highProfileDistribution
         numberOfInteractions = np.random.choice(range(len(profileDistribution)), p=profileDistribution)
         if numberOfInteractions == 0 :
            continue
         
         for interaction in np.random.choice(newInteractions, size=numberOfInteractions, replace=False) :
            interactionGraph.add_interaction(node, interaction, t=timestamp)

         
                  
   #create an empty Interaction Graph
   interactionGraph = dn.DynGraph()
   
   #interactions at timetsamp t = 0
   #interactions based on a scale-free network model
   G = nx.barabasi_albert_graph(n=preferenceGraph.number_of_nodes(), m=3)
   interactionGraph.add_interactions_from(G.edges(), t=0)
   

   #interactions at timestamp t = 1
   #interactions based on Preference Graph (students' preferences for their classmates)
   edgesProbabilities = analyzePreferenceGraph(timestamp=1)


   #analyze students' preferences
   #Based on the personalityThreshold, split students in 2-categories: low profile and high profile
   lowProfileStudents, highProfileStudents, lowProfileDistribution, highProfileDistribution = analyzeStudentsPreferences(timestamp=1)
   
   
   #interactions at timestamp t = 2
   createRandomEdgesAtTimestamp(2)
   
   
    
   return interactionGraph





path = "./preferenceGraph.json"
interactionGraph(preferenceGraph=readPreferenceGraph(path), personalityThreshold=0.4)


49
48
47
47
48
48
49
48
49
49
49
48
48
48
48
47
48
49
47
46
46
46
49
48
48
49
46
48
48
48
48
49
48
47
49
48
48
48
49
46
49
48
48
47
49
49
48
49
49
48


In [ ]:
#take into account the classmates that the nodes want to avoid
#take into maximum number of interactions

In [152]:
arr = poisson.pmf(range(5), mu=1.5)